# Import Libraries

In [24]:
import numpy as np
import pandas as pd
from hmmlearn import hmm
import warnings
from constants import *
import math
import random
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

# Helper Methods

In [25]:
def generate_random_sample(X, size):
    '''
    Given a list X, 
    generate random samples of given size
    '''
    Z_temp = random.sample(list(X), size)
    
    #Concatenation
    Z = [Z_temp[0]]
    for val in Z_temp[1:]:
        Z = np.concatenate([Z,[val]])
    
    return Z

def convert_values_to_list(list_val):
    '''
    Given a list X = [1 2 3] , 
    return X = [[1],[2],[3]]
    '''
    X = []
    for i in list_val:
        X.append([i])
    return X

def conversion_list_of_list(X, DIMENSION):
    '''
    Given a list X with values in lists,
    X = [[ 3  2  2],[4 8 10]]
    Convert each value to a list
    Return list of lists, array of lengths of each sequence
    X = [[3] [2] [2] [4] [8] [10]]
    ''' 
    X_new = []
    length = len(X)
    for idx, val_list in enumerate(X):
        Y = []
        for val in val_list:
            Y.append([val])
        X_new.append(Y)

    #Concatenation
    Z = X_new[0]
    for val_list in X_new[1:]:
        Z = np.concatenate([Z,val_list])

    # assign array of lengths for HMM
    lengths = [DIMENSION]*length
    
    return Z,lengths

#Calculate likelihood for given sequence according to given HMMs and return HMM
def likelihood_sequence(sequence, HMM_array):
    '''
    Given list of K HMMs and sequence,
    determines likelihood of sequence under all HMM models
    Returns index of HMM which has max likelihood
    ''' 
    scores = []
    length = [len(sequence)]
    for i, HMM in enumerate(HMM_array):
        calculated_score = HMM.score(sequence, length)
        scores.append(calculated_score)
    idx = scores.index(max(scores))
    return idx

def HMM_model_stats(model):
    '''
    Details of HMM model
    ''' 
    print("*************************************")
    print("Transition matrix")
    print(model.transmat_)
    print("*************************************")
    print("Means and stds of each hidden state")
    for i in range(model.n_components):
        print("Hidden state {0}".format(i))
        print("mean = ", model.means_[i])
        print("std = ", [math.sqrt(model.covars_[i])])
        print()

def BIC(HMM,X):
    LogLikelihood = model.score(X)
    num_hidden_states = model.n_components
    # D counts transition matrix, emission matrix, sequences estimated (Z), covariance matrix
    D = (num_hidden_states) + 2*(num_hidden_states**2) + len(X)*DIMENSION 
    BIC = LogLikelihood - (D/2)*np.log(len(X))
    return BIC
    
def BIC_array(HMM_array,X_i):
    BIC_total = 0
    for i in range(len(X_i)):
        model = HMM_array[i]
        X, lengths = conversion_list_of_list(X_i[i],DIMENSION)
        LogLikelihood = model.score(X,lengths)
        num_hidden_states = model.n_components
        # D counts transition matrix, emission matrix, sequences estimated (Z), covariance matrix
        D = num_hidden_states + 2*(num_hidden_states**2) + len(X)*DIMENSION 
        BIC = LogLikelihood - (D/2)*np.log(len(X))
        BIC_total+= BIC
    return BIC_total

def likelihood_array(HMM_array,X_i):
    likelihood_total = 0
    for i in range(len(X_i)):
        model = HMM_array[i]
        X, lengths = conversion_list_of_list(X_i[i],DIMENSION)
        LogLikelihood = model.score(X,lengths)
        likelihood = LogLikelihood 
        likelihood_total+= likelihood
    return likelihood_total

def plot_BIC(list_k, BIC_score):
    fig = plt.subplot(111)
    plt.plot(list_k, BIC_score, marker='o')  
    plt.xlabel('Value of K')
    plt.ylabel('Objective')
    plt.title('BIC')
    plt.show() 

# Load Data and Clean

In [26]:
df = pd.read_csv('Data/merged.txt', sep=",", na_values=['-'])
df = df.dropna()
df = df[['cdReads0','cdReads1','cdReads2','cdRPKM0','cdRPKM1','cdRPKM2']]


# Filter cdReads

In [27]:
df = df[(df['cdReads0'] >= 50) & (df['cdReads1'] >= 50) & (df['cdReads2'] >= 50)]

In [28]:
#Dataset
df_main = df[['cdReads0','cdReads1','cdReads2']]
LENGTH,DIMENSION = df_main.shape
print("Dataset size is",LENGTH)
print("Features are", DIMENSION)
print(df_main.head(5))
X = np.log2(df_main.values)
print("****************************")
print("First 5 log2 values\n",X[:5])

Dataset size is 2072
Features are 3
    cdReads0  cdReads1  cdReads2
6      113.0     180.0     292.0
19     455.0     340.0     326.0
34     353.0     397.0     499.0
57     113.0     150.0     137.0
81     126.0     175.0     202.0
****************************
First 5 log2 values
 [[ 6.82017896  7.4918531   8.18982456]
 [ 8.82972274  8.40939094  8.34872815]
 [ 8.46352437  8.6329952   8.96289601]
 [ 6.82017896  7.22881869  7.09803208]
 [ 6.97727992  7.45121111  7.65821148]]


# Arrays with HMM models for 1<=K<=25

In [29]:
HMM_K_ARRAYS = []
X_i_K_ARRAYS = []

# Check likelihood and do assignments

In [30]:
K_values = range(2,25)

In [31]:
for K in K_values:
    HMM_array = []
    X_i = []
    print("**************** K = ",K,"************************")
    for i in range(K):
        X_i.append([])

    # Sequences for initial HMM estimation
    for i in range(LENGTH):
        for j in range(K):
            if(i%K==j):
                X_i[j].append(list(X[i]))
    
    NUM_ITERATIONS = 0
    NUM_CLUSTER_PREV = {}
    NUM_CLUSTER_NOW = {}
                
    for i in range(K):
        model = hmm.GaussianHMM(n_components=3,covariance_type='spherical')
        X_temp, lengths = conversion_list_of_list(X_i[i],DIMENSION)
        model.fit(X_temp, lengths)
        HMM_array.append(model)
    
    likelihood_prev = likelihood_array(HMM_array,X_i)
    print("Likelihood for iteration",NUM_ITERATIONS,"is",likelihood_prev)
    NUM_ITERATIONS+=1
    while (True):
        # Assign all sequences to HMM models

        print("************ Check likelihood of sequence in HMM  *********")
        NUM_CLUSTER_NOW = {}
        for x in X:
            sequence = convert_values_to_list(x)
            hmm_index = likelihood_sequence(sequence, HMM_array)
            X_i[hmm_index].append(list(x))
            if (hmm_index not in NUM_CLUSTER_NOW):
                NUM_CLUSTER_NOW[hmm_index] = 1
            else:
                NUM_CLUSTER_NOW[hmm_index] += 1
        print("************ Checking likelihood done  *********")


        # Re-estimate parameters for new HMMs
        print("************ Re-estimating HMM *********")
        HMM_array = []
        for i in range(K):
            model = hmm.GaussianHMM(n_components=3,covariance_type='spherical')
            X_temp, lengths = conversion_list_of_list(X_i[i], DIMENSION)
            model.fit(X_temp, lengths)
            HMM_array.append(model)
        print("************ Re-estimation done *********")
        print("Previous assignments",NUM_CLUSTER_PREV)
        print("Current assignments",NUM_CLUSTER_NOW)
        likelihood_curr = likelihood_array(HMM_array,X_i)
        print("Likelihood for iteration",NUM_ITERATIONS,"is",likelihood_curr)
        print("*****************************************")

        # if no reassignments, then break
        if ((NUM_CLUSTER_PREV == NUM_CLUSTER_NOW)):
            HMM_K_ARRAYS.append(HMM_array)
            X_i_K_ARRAYS.append(X_i)
            break
        else:
            # initialize empty subsets of data for next iteration
            X_i = []
            for i in range(K):
                X_i.append([])

            NUM_CLUSTER_PREV = NUM_CLUSTER_NOW
            print("Num iterations is:", NUM_ITERATIONS)
            NUM_ITERATIONS += 1
            likelihood_prev = likelihood_curr
    print("**********************************************************\n\n")

**************** K =  2 ************************
Likelihood for iteration 0 is -5654.61810203
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {}
Current assignments {0: 1206, 1: 866}
Likelihood for iteration 1 is -10553.2642738
*****************************************
Num iterations is: 1
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {0: 1206, 1: 866}
Current assignments {0: 1140, 1: 932}
Likelihood for iteration 2 is -4130.14019064
*****************************************
Num iterations is: 2
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-e

************ Re-estimation done *********
Previous assignments {1: 1058, 0: 1014}
Current assignments {1: 1059, 0: 1013}
Likelihood for iteration 22 is -3729.2479998
*****************************************
Num iterations is: 22
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {1: 1059, 0: 1013}
Current assignments {1: 1059, 0: 1013}
Likelihood for iteration 23 is -3729.2479998
*****************************************
**********************************************************


**************** K =  3 ************************
Likelihood for iteration 0 is -5655.19163873
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {}
Current assignments {0: 1236, 2: 460, 1: 

************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {1: 796, 0: 602, 2: 674}
Current assignments {1: 796, 0: 603, 2: 673}
Likelihood for iteration 20 is -2806.17741204
*****************************************
Num iterations is: 20
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {1: 796, 0: 603, 2: 673}
Current assignments {1: 796, 0: 606, 2: 670}
Likelihood for iteration 21 is -2806.52844545
*****************************************
Num iterations is: 21
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {1: 796, 0: 606, 2: 670}
Current assignments {1: 797, 0: 607,

************ Re-estimation done *********
Previous assignments {0: 472, 2: 473, 3: 497, 1: 630}
Current assignments {0: 478, 2: 472, 3: 496, 1: 626}
Likelihood for iteration 15 is -2276.86445852
*****************************************
Num iterations is: 15
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {0: 478, 2: 472, 3: 496, 1: 626}
Current assignments {0: 480, 2: 470, 3: 501, 1: 621}
Likelihood for iteration 16 is -2272.60396799
*****************************************
Num iterations is: 16
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {0: 480, 2: 470, 3: 501, 1: 621}
Current assignments {0: 483, 2: 468, 3: 506, 1: 615}
Likelihood for iteration 17 is -2

Likelihood for iteration 35 is -2151.33824928
*****************************************
Num iterations is: 35
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {0: 484, 2: 566, 3: 481, 1: 541}
Current assignments {0: 468, 2: 585, 3: 479, 1: 540}
Likelihood for iteration 36 is -2143.56429692
*****************************************
Num iterations is: 36
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {0: 468, 2: 585, 3: 479, 1: 540}
Current assignments {0: 458, 2: 597, 3: 477, 1: 540}
Likelihood for iteration 37 is -2137.43874453
*****************************************
Num iterations is: 37
************ Check likelihood of sequence in HMM  *********
************

...

Likelihood for iteration 37 is 700.454720226
*****************************************
Num iterations is: 37
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {12: 122, 6: 108, 2: 107, 14: 147, 10: 168, 5: 126, 16: 158, 0: 153, 9: 123, 7: 131, 4: 217, 17: 181, 1: 114, 11: 98, 8: 2, 3: 114, 13: 2, 15: 1}
Current assignments {12: 122, 6: 108, 2: 107, 14: 147, 10: 169, 5: 127, 16: 160, 0: 153, 9: 123, 7: 131, 4: 213, 17: 181, 1: 113, 11: 99, 8: 2, 3: 114, 13: 2, 15: 1}
Likelihood for iteration 38 is 701.249859528
*****************************************
Num iterations is: 38
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {12: 122, 6: 108, 2: 107, 14: 147, 10: 169, 

Likelihood for iteration 50 is 713.542720655
*****************************************
Num iterations is: 50
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {12: 122, 6: 108, 2: 107, 14: 157, 10: 164, 5: 124, 16: 165, 0: 147, 9: 127, 7: 137, 4: 182, 17: 181, 1: 115, 11: 103, 8: 2, 3: 128, 13: 2, 15: 1}
Current assignments {12: 122, 6: 108, 2: 107, 14: 160, 10: 164, 5: 124, 16: 165, 0: 145, 9: 128, 7: 138, 4: 182, 17: 180, 1: 112, 11: 103, 8: 2, 3: 129, 13: 2, 15: 1}
Likelihood for iteration 51 is 715.556659297
*****************************************
Num iterations is: 51
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {12: 122, 6: 108, 2: 107, 14: 160, 10: 164

Likelihood for iteration 63 is 718.972876866
*****************************************
Num iterations is: 63
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {12: 122, 6: 108, 2: 107, 14: 163, 10: 165, 5: 128, 16: 167, 0: 147, 9: 129, 7: 143, 4: 176, 17: 169, 1: 112, 11: 103, 8: 2, 3: 128, 13: 2, 15: 1}
Current assignments {12: 122, 6: 108, 2: 107, 14: 163, 10: 165, 5: 128, 16: 168, 0: 148, 9: 129, 7: 143, 4: 176, 17: 168, 1: 112, 11: 103, 8: 2, 3: 127, 13: 2, 15: 1}
Likelihood for iteration 64 is 718.664100763
*****************************************
Num iterations is: 64
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {12: 122, 6: 108, 2: 107, 14: 163, 10: 165

Likelihood for iteration 76 is 763.211330187
*****************************************
Num iterations is: 76
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {12: 122, 6: 108, 2: 107, 14: 164, 10: 165, 5: 117, 16: 178, 0: 147, 9: 129, 7: 120, 4: 182, 17: 183, 1: 112, 11: 103, 8: 2, 3: 130, 13: 2, 15: 1}
Current assignments {12: 122, 6: 108, 2: 107, 14: 164, 10: 165, 5: 116, 16: 179, 0: 147, 9: 129, 7: 119, 4: 185, 17: 183, 1: 112, 11: 103, 8: 2, 3: 128, 13: 2, 15: 1}
Likelihood for iteration 77 is 767.440218567
*****************************************
Num iterations is: 77
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {12: 122, 6: 108, 2: 107, 14: 164, 10: 165

IndexError: list index out of range

In [33]:
import dill
dill.dump_session('notebook_env.db')

# Calculate BIC for these assignments

In [ ]:
print(BIC_array(HMM_array,X_i))

In [ ]:
print(BIC_array(HMM_array,X_i))

# Noise Cluster

In [ ]:
# Get list of lists to fit
Z, lengths = conversion_list_of_list(X,DIMENSION)

In [ ]:
# Noise cluster 

model_noise = hmm.GaussianHMM(n_components=3,covariance_type='spherical')
model_noise.fit(Z, lengths)

In [ ]:
HMM_model_stats(model_noise)

# Generate subsets of data for K HMMs

In [8]:
K = 3
HMM_array = []
X_i = []

for i in range(K):
    X_i.append([])

# Sequences for initial HMM estimation
for i in range(LENGTH):
    for j in range(K):
        if(i%K==j):
            X_i[j].append(list(X[i]))

In [ ]:
for i in range(K):
    model = hmm.GaussianHMM(n_components=3,covariance_type='spherical')
    X_temp, lengths = conversion_list_of_list(X_i[i],DIMENSION)
    model.fit(X_temp, lengths)
    HMM_array.append(model)

In [ ]:
likelihood_prev = likelihood_array(HMM_array,X_i)
print("Likelihood for iteration",0,"is",likelihood_prev)

## Statistics for K HMMs

In [ ]:
for i in range(K):
    print("Statistics for HMM Model", i)
    HMM_model_stats(HMM_array[i])
    print('\n\n')

In [ ]:
NUM_ITERATIONS = 1
NUM_CLUSTER_PREV = {}
NUM_CLUSTER_NOW = {}

In [ ]:
#initialize empty subsets of data
X_i = []

for i in range(K):
    X_i.append([])